# FastPitch Adapter Finetuning

This notebook is designed to provide a guide on how to run FastPitch Adapter Finetuning Pipeline. It contains the following sections:
1. **Fine-tune FastPitch on adaptation data**: fine-tune pre-trained multi-speaker FastPitch for a new speaker
* Dataset Preparation: download dataset and extract manifest files. (duration more than 15 mins)
* Preprocessing: add absolute audio paths in manifest and extract Supplementary Data.
* **Model Setting: transform pre-trained checkpoint to adapter-compatible checkpoint and precompute speaker embedding**
* Training: fine-tune frozen multispeaker FastPitch with trainable adapters.
2. **Fine-tune HiFiGAN on adaptation data**: fine-tune a vocoder for the fine-tuned multi-speaker FastPitch
* Dataset Preparation: extract mel-spectrograms from fine-tuned FastPitch.
* Training: fine-tune HiFiGAN with fine-tuned adaptation data.
3. **Inference**: generate speech from adapted FastPitch
* Load Model: load pre-trained multi-speaker FastPitch with **fine-tuned adapters**.
* Output Audio: generate audio files.

# License

> Copyright (c) 2023, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [ ]:
"""
You can either run this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.
Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies# .
"""
# # If you're using Colab and not running locally, uncomment and run this cell.
# BRANCH = 'r1.22.0'
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode pynini==2.1.4 scipy==1.7.3
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# # Download local version of NeMo scripts. If you are running locally and want to use your own local NeMo code,
# # comment out the below lines and set `code_dir` to your local path.
code_dir = 'NeMoTTS' 
!git clone https://github.com/NVIDIA/NeMo.git {code_dir}

In [ ]:
!wandb login #PASTE_WANDB_APIKEY_HERE

## Set finetuning params

This notebook expects a pretrained model to finetune. If you have a pretrained multispeaker checkpoint, set the path in next block to the path of pretrained checkpoint. You can also pretrain a multispeaker adapter checkpoint using the [FastPitch_MultiSpeaker_Pretraining tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/FastPitch_MultiSpeaker_Pretraining.ipynb).

In [ ]:
# .nemo files for your pre-trained FastPitch and HiFiGAN
pretrained_fastpitch_checkpoint = "<Multispeaker pretrained checkpoint path.>"
finetuned_hifigan_on_multispeaker_checkpoint = "<Pretrained hifiGan checkpoint path.>"
use_ipa = False #Set to False while using Arpabet.

In [ ]:
sample_rate = 44100
# Store all manifest and audios
data_dir = 'NeMoTTS_dataset'
# Store all supplementary files
supp_dir = "NeMoTTS_sup_data"
# Store all training logs
logs_dir = "NeMoTTS_logs"
# Store all mel-spectrograms for vocoder training
mels_dir = "NeMoTTS_mels"

In [ ]:
import os
import json
import shutil
import nemo
import torch
import numpy as np

from pathlib import Path
from tqdm import tqdm

In [ ]:
os.makedirs(code_dir, exist_ok=True)
code_dir = os.path.abspath(code_dir)
os.makedirs(data_dir, exist_ok=True)
data_dir = os.path.abspath(data_dir)
os.makedirs(supp_dir, exist_ok=True)
supp_dir = os.path.abspath(supp_dir)
os.makedirs(logs_dir, exist_ok=True)
logs_dir = os.path.abspath(logs_dir)
os.makedirs(mels_dir, exist_ok=True)
mels_dir = os.path.abspath(mels_dir)

# 1. Fine-tune FastPitch on adaptation data

## a. Data Preparation
For our tutorial, we use small part of VCTK dataset with a new target speaker (p267). Usually, the audios should have total duration more than 15 minutes.

In [ ]:
!cd {data_dir} && wget https://vctk-subset.s3.amazonaws.com/vctk_subset.tar.gz && tar zxf vctk_subset.tar.gz

In [ ]:
manidir = f"{data_dir}/vctk_subset"
!ls {manidir}

In [ ]:
train_manifest = os.path.abspath(os.path.join(manidir, 'train.json'))
valid_manifest = os.path.abspath(os.path.join(manidir, 'dev.json'))

## b. Preprocessing

### Add absolute file path in manifest
We use absolute path for audio_filepath to get the audio during training.

In [ ]:
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest

In [ ]:
train_data = read_manifest(train_manifest)
for m in train_data: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
write_manifest(train_manifest, train_data)

valid_data = read_manifest(valid_manifest)
for m in valid_data: m['audio_filepath'] = os.path.abspath(os.path.join(manidir, m['audio_filepath']))
write_manifest(valid_manifest, valid_data)

### Extract Supplementary Data

As mentioned in the [FastPitch and MixerTTS training tutorial](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/FastPitch_MixerTTS_Training.ipynb) - To accelerate and stabilize our training, we also need to extract pitch for every audio, estimate pitch statistics (mean, std, min, and max). To do this, all we need to do is iterate over our data one time, via `extract_sup_data.py` script.

In [ ]:
!cd {code_dir} && python scripts/dataset_processing/tts/extract_sup_data.py \
    manifest_filepath={train_manifest} \
    sup_data_path={supp_dir} \
    dataset.sample_rate={sample_rate} \
    dataset.n_fft=2048 \
    dataset.win_length=2048 \
    dataset.hop_length=512

After running the above command line, you will observe a new folder NeMoTTS_sup_data/pitch and printouts of pitch statistics like below. Specify these values to the FastPitch training configurations. We will be there in the following section.
```bash
PITCH_MEAN=175.48513793945312, PITCH_STD=42.3786735534668
PITCH_MIN=65.4063949584961, PITCH_MAX=270.8517761230469
```

In [ ]:
!cd {code_dir} && python scripts/dataset_processing/tts/extract_sup_data.py \
    manifest_filepath={valid_manifest} \
    sup_data_path={supp_dir} \
    dataset.sample_rate={sample_rate} \
    dataset.n_fft=2048 \
    dataset.win_length=2048 \
    dataset.hop_length=512

## c. Model Setting
### Transform pre-trained checkpoint to adapter-compatible checkpoint

In [ ]:
from nemo.collections.tts.models import FastPitchModel
from nemo.collections.asr.parts.preprocessing.features import WaveformFeaturizer
from nemo.core import adapter_mixins
from omegaconf import DictConfig, OmegaConf, open_dict

In [ ]:
def update_model_config_to_support_adapter(config) -> DictConfig:
    with open_dict(config):
        enc_adapter_metadata = adapter_mixins.get_registered_adapter(config.input_fft._target_)
        if enc_adapter_metadata is not None:
            config.input_fft._target_ = enc_adapter_metadata.adapter_class_path

        dec_adapter_metadata = adapter_mixins.get_registered_adapter(config.output_fft._target_)
        if dec_adapter_metadata is not None:
            config.output_fft._target_ = dec_adapter_metadata.adapter_class_path

        pitch_predictor_adapter_metadata = adapter_mixins.get_registered_adapter(config.pitch_predictor._target_)
        if pitch_predictor_adapter_metadata is not None:
            config.pitch_predictor._target_ = pitch_predictor_adapter_metadata.adapter_class_path

        duration_predictor_adapter_metadata = adapter_mixins.get_registered_adapter(config.duration_predictor._target_)
        if duration_predictor_adapter_metadata is not None:
            config.duration_predictor._target_ = duration_predictor_adapter_metadata.adapter_class_path

        aligner_adapter_metadata = adapter_mixins.get_registered_adapter(config.alignment_module._target_)
        if aligner_adapter_metadata is not None:
            config.alignment_module._target_ = aligner_adapter_metadata.adapter_class_path

    return config

In [ ]:
spec_model = FastPitchModel.restore_from(pretrained_fastpitch_checkpoint).eval().cuda()
spec_model.cfg = update_model_config_to_support_adapter(spec_model.cfg)

### Precompute Speaker Embedding
Get all GST speaker embeddings from training data, take average, and save as `precomputed_emb` in FastPitch

In [ ]:
wave_model = WaveformFeaturizer(sample_rate=sample_rate)
train_data = read_manifest(train_manifest)

spk_embs = []  
for data in train_data:
    with torch.no_grad():
        audio = wave_model.process(data['audio_filepath'])
        audio_length = torch.tensor(audio.shape[0]).long()
        audio = audio.unsqueeze(0).to(device=spec_model.device)
        audio_length = audio_length.unsqueeze(0).to(device=spec_model.device)
        spec_ref, spec_ref_lens = spec_model.preprocessor(input_signal=audio, length=audio_length)
        spk_emb = spec_model.fastpitch.get_speaker_embedding(batch_size=spec_ref.shape[0],
                                                             speaker=None,
                                                             reference_spec=spec_ref,
                                                             reference_spec_lens=spec_ref_lens)

    spk_embs.append(spk_emb.squeeze().cpu())

spk_embs = torch.stack(spk_embs, dim=0)
spk_emb  = torch.mean(spk_embs, dim=0)
spk_emb_dim = spk_emb.shape[0]

with open_dict(spec_model.cfg):
    spec_model.cfg.speaker_encoder.precomputed_embedding_dim = spec_model.cfg.symbols_embedding_dim

spec_model.fastpitch.speaker_encoder.overwrite_precomputed_emb(spk_emb)

In [ ]:
spec_model.save_to('Pretrained-FastPitch.nemo')
shutil.copyfile(finetuned_hifigan_on_multispeaker_checkpoint, "Pretrained-HifiGan.nemo")
pretrained_fastpitch_checkpoint = os.path.abspath("Pretrained-FastPitch.nemo")
finetuned_hifigan_on_multispeaker_checkpoint = os.path.abspath("Pretrained-HifiGan.nemo")

## d. Training

In [ ]:
phone_dict_name = "ipa_cmudict-0.7b_nv23.01.txt" if use_ipa else "cmudict-0.7b_nv22.10"
phoneme_dict_path = os.path.abspath(os.path.join(code_dir, "scripts", "tts_dataset_files", phone_dict_name))
heteronyms_path = os.path.abspath(os.path.join(code_dir, "scripts", "tts_dataset_files", "heteronyms-052722"))

# Copy and Paste the PITCH_MEAN and PITCH_STD from previous steps (train_manifest) to override pitch_mean and pitch_std configs below.
PITCH_MEAN=175.48513793945312
PITCH_STD=42.3786735534668

config_filename = "fastpitch_align_ipa_adapter.yaml" if use_ipa else "fastpitch_align_44100_adapter.yaml"

### Important notes
* `+init_from_nemo_model`: initialize with a multi-speaker FastPitch checkpoint
* `model.speaker_encoder.precomputed_embedding_dim={spk_emb_dim}`: use precomputed speaker embedding
* `~model.speaker_encoder.lookup_module`: we use precomputed speaker embedding, so we remove the pre-trained looked-up speaker embedding
* `~model.speaker_encoder.gst_module`:  we use precomputed speaker embedding, so we remove the pre-trained gst speaker embedding
* Other optional arguments based on your preference:
    * batch_size
    * exp_manager
    * trainer
    * model.unfreeze_aligner=true
    * model.unfreeze_duration_predictor=true
    * model.unfreeze_pitch_predictor=true

In [ ]:
# Normally 200 epochs
!cd {code_dir} && python examples/tts/fastpitch_finetune_adapters.py \
--config-name={config_filename} \
+init_from_nemo_model={pretrained_fastpitch_checkpoint} \
train_dataset={train_manifest} \
validation_datasets={valid_manifest} \
sup_data_types="['align_prior_matrix', 'pitch', 'energy']" \
sup_data_path={supp_dir} \
pitch_mean={PITCH_MEAN} \
pitch_std={PITCH_STD} \
model.speaker_encoder.precomputed_embedding_dim={spk_emb_dim} \
~model.speaker_encoder.lookup_module \
~model.speaker_encoder.gst_module \
model.train_ds.dataloader_params.batch_size=8 \
model.validation_ds.dataloader_params.batch_size=8 \
+model.text_tokenizer.add_blank_at=True \
model.optim.name=adam \
model.optim.lr=1e-3 \
model.optim.sched.warmup_steps=0 \
+model.optim.sched.min_lr=1e-4 \
exp_manager.exp_dir={logs_dir} \
+exp_manager.create_wandb_logger=True \
+exp_manager.wandb_logger_kwargs.name="tutorial-FastPitch-finetune-adaptation" \
+exp_manager.wandb_logger_kwargs.project="NeMo" \
+exp_manager.checkpoint_callback_params.save_top_k=-1 \
trainer.max_epochs=20 \
trainer.check_val_every_n_epoch=10 \
trainer.log_every_n_steps=1 \
trainer.devices=1 \
trainer.strategy=ddp \
trainer.precision=32

In [ ]:
# e.g. NeMoTTS_logs/FastPitch/Y-M-D_H-M-S/checkpoints/FastPitch.nemo
# e.g. NeMoTTS_logs/FastPitch/Y-M-D_H-M-S/checkpoints/adapters.pt
last_checkpoint_dir = sorted(list([i for i in (Path(logs_dir) / "FastPitch").iterdir() if i.is_dir()]))[-1] / "checkpoints"
finetuned_fastpitch_checkpoint = list(last_checkpoint_dir.glob('*.nemo'))[0]
finetuned_adapter_checkpoint = list(last_checkpoint_dir.glob('adapters.pt'))[0]
print(finetuned_fastpitch_checkpoint)
print(finetuned_adapter_checkpoint)

# 3. Fine-tune HiFiGAN on adaptation data

## a. Dataset Preparation
Generate mel-spectrograms for HiFiGAN training.

In [ ]:
!cd {code_dir} \
&& python scripts/dataset_processing/tts/resynthesize_dataset.py \
--model-path={finetuned_fastpitch_checkpoint} \
--input-json-manifest={train_manifest} \
--input-sup-data-path={supp_dir} \
--output-folder={mels_dir} \
--device="cuda:0" \
--batch-size=1 \
--num-workers=1 \
&& python scripts/dataset_processing/tts/resynthesize_dataset.py \
--model-path={finetuned_fastpitch_checkpoint} \
--input-json-manifest={valid_manifest} \
--input-sup-data-path={supp_dir} \
--output-folder={mels_dir} \
--device="cuda:0" \
--batch-size=1 \
--num-workers=1

In [ ]:
train_manifest_mel = f"{mels_dir}/train_mel.json"
valid_manifest_mel = f"{mels_dir}/dev_mel.json"

## b. Training

In [ ]:
# Normally 500 epochs
!cd {code_dir} && python examples/tts/hifigan_finetune.py \
--config-name=hifigan_44100.yaml \
train_dataset={train_manifest_mel} \
validation_datasets={valid_manifest_mel} \
+init_from_nemo_model={finetuned_hifigan_on_multispeaker_checkpoint} \
model.train_ds.dataloader_params.batch_size=32 \
model.optim.lr=0.0001 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune \
+trainer.max_epochs=50 \
trainer.check_val_every_n_epoch=5 \
trainer.devices=-1 \
trainer.strategy='ddp_find_unused_parameters_true' \
trainer.precision=16 \
exp_manager.exp_dir={logs_dir} \
exp_manager.create_wandb_logger=True \
exp_manager.wandb_logger_kwargs.name="tutorial-HiFiGAN-finetune-multispeaker" \
exp_manager.wandb_logger_kwargs.project="NeMo"

In [ ]:
# e.g. NeMoTTS_logs/HifiGan/Y-M-D_H-M-S/checkpoints/HifiGan.nemo
last_checkpoint_dir = sorted(list([i for i in (Path(logs_dir) / "HifiGan").iterdir() if i.is_dir()]))[-1] / "checkpoints"
finetuned_hifigan_on_adaptation_checkpoint = list(last_checkpoint_dir.glob('*.nemo'))[0]
finetuned_hifigan_on_adaptation_checkpoint

# 4. Inference

In [ ]:
from nemo.collections.tts.models import HifiGanModel
import IPython.display as ipd
import matplotlib.pyplot as plt

## a. Load Model

In [ ]:
# Load from pretrained FastPitch and finetuned adapter
# spec_model = FastPitchModel.restore_from(pretrained_fastpitch_checkpoint)
# spec_model.load_adapters(finetuned_adapter_checkpoint)

# Load from finetuned FastPitch
spec_model = FastPitchModel.restore_from(finetuned_fastpitch_checkpoint)
spec_model = spec_model.eval().cuda()

In [ ]:
# HiFiGAN
vocoder_model = HifiGanModel.restore_from(finetuned_hifigan_on_adaptation_checkpoint).eval().cuda()

## b. Output Audio

In [ ]:
def gen_spectrogram(text, spec_gen_model):
    parsed = spec_gen_model.parse(text)
    with torch.no_grad():    
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed)
    return spectrogram
  
def synth_audio(vocoder_model, spectrogram):    
    with torch.no_grad():  
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return audio

In [ ]:
# Validatation Audio
num_val = 3
val_records = []
with open(valid_manifest, "r") as f:
    for i, line in enumerate(f):
        val_records.append(json.loads(line))
        if len(val_records) >= num_val:
            break

In [ ]:
for i, val_record in enumerate(val_records):
    spec_pred = gen_spectrogram(val_record['text'], spec_model)
    audio_gen = synth_audio(vocoder_model, spec_pred)

    audio_gt = ipd.Audio(val_record['audio_filepath'], rate=sample_rate)
    audio_gen = ipd.Audio(audio_gen, rate=sample_rate)
    
    print("------")
    print(f"Text: {val_record['text']}")
    print('Ground Truth Audio')
    ipd.display(audio_gt)
    print('Synthesized Audio')
    ipd.display(audio_gen)
    plt.imshow(spec_pred[0].to('cpu').numpy(), origin="lower", aspect="auto")
    plt.show()

In [ ]:
print(f"Finetuned FastPitch: {finetuned_fastpitch_checkpoint}")
print(f"Finetuned HiFi-Gan: {finetuned_hifigan_on_adaptation_checkpoint}")